In [ ]:
# Python MP3 Music Player Program
#
# This program will play music from MP3 files. It will also display
# the album cover, song title, and artist name for each song.
#
#
# Instructions:
# Place MP3 files into the 'song-folder' folder. MP3 files must have
# Title, Album and Contributing Artist properties filled in to display 
# the correct information. 

import pygame
import os,sys
import eyed3
from tkinter import *
from PIL import Image, ImageTk
from get_cover_art import CoverFinder

def build_program():
    """
    Function to build the interface and components.
    """
    # Initialize PyGame and TKinter
    global root
    pygame.init()
    root = Tk()

    root.geometry("414x659")
    root.title("Python MP3 Music Player - by ")
    
    
    # Create the playlist, fill the playlist
    global playlist
    playlist = Listbox(root, bg="black", fg="white", width=65)
    playlist.grid(row = 1, pady=20)

    song_list = os.listdir("song-folder/")
    for song in song_list:
        if (song.endswith(".mp3")):
            song = song.replace(".mp3", "")
            playlist.insert(END, song)
            
            
    # Create reload songlist and exit menu
    mainMenu = Menu(root)
    root.config(menu = mainMenu)
    reloadMenu = Menu(mainMenu)
    mainMenu.add_cascade(label = "Reload", menu=reloadMenu)
    reloadMenu.add_command(label= "Reload Playlist", command=reloadPlaylist)
    
    exitMenu = Menu(mainMenu)
    mainMenu.add_cascade(label = "Exit", menu=exitMenu)
    exitMenu.add_command(label = "Exit Program", command=exit)
    
    # Create labels and images
    defaultImage = Image.open('song-folder/art-default')
    
    global photo
    global albumArtwork
    global songTitle
    global artistName
    
    photo = ImageTk.PhotoImage(defaultImage)
    albumArtwork = Label(image = photo)
    albumArtwork.image = photo
    albumArtwork.grid(row = 2)
    
    someSpace = Label(text = "")
    moreSpace = Label(text = "")
    songTitle = Label(text = 'Song Title')
    artistName = Label(text = 'Artist Name')
    someSpace.grid(row=3)
    songTitle.grid(row=4)
    artistName.grid(row=5)
    moreSpace.grid(row=6)

    # Create control buttons in their own frame
    buttons = Frame(root)
    buttons.grid()

    button_back = Button(buttons, text = "<<", command=playLast)
    button_play = Button(buttons, text = "|>", command=playTrack)
    button_pause = Button(buttons, text = "||", command=pauseTrack)
    button_forward = Button(buttons, text = ">>", command=playNext)
    
    button_back.grid(row=7, column=0, padx=4)
    button_play.grid(row=7, column=1, padx=4)
    button_pause.grid(row=7, column=2, padx=4)
    button_forward.grid(row=7, column=3, padx=4)
    root.mainloop()

def exit():
    """
    Function to exit the entire program.
    """
    pygame.quit()
    root.destroy()

def reloadPlaylist():
    """
    Function to reload the playlist for new songs
    """
    global playlist
    pygame.mixer.music.stop()
    playlist.selection_clear(0, END)
    playlist.delete(0,END)
    song_list = os.listdir("song-folder/")
    for song in song_list:
        if (song.endswith(".mp3")):
            song = song.replace(".mp3", "")
            playlist.insert(END, song)
    
    getAlbumArt()
    
def getAlbumArt():
    """
    Function to download album art.
    """
    # never embed into audio file, always force to look for album art, always 256
    finder = CoverFinder({'no-embed':True,
                          'force':True,
                          'art-size':256})
    song_list = os.listdir("song-folder/")
    for song in song_list:
        if (song.endswith(".mp3")):
            finder.scan_file(f'song-folder/{song}')       


global isPaused
isPaused = False
    
def playTrack():
    """
    Function to play and resume the track.
    """
    # resume if it is just paused
    global isPaused
    if isPaused:
        pygame.mixer.music.unpause()
        isPaused = False
    else:
        if(playlist.curselection() == ()): # no selection? start at beginning
            playlist.activate(0)
            playlist.selection_set(0, last=None)
        song = playlist.get(ACTIVE) # play selected track
        song = f'song-folder/{song}.mp3'
        pygame.mixer.music.load(song)
        pygame.mixer.music.play(loops=0)
        isPaused = False

    setSongDisplay(playlist.get(ACTIVE))


def pauseTrack():
    """
    Function to pause and unpause the track.
    """
    global isPaused
    isPaused = True
    pygame.mixer.music.pause()
        
        
def playNext():
    """
    Function to play the next track.
    """
    # select next song in song box, select first in list if at the end
    if(playlist.get(ACTIVE) != playlist.get(END)):
        nextSong = playlist.curselection()
        nextSong = nextSong[0]+1
        playlist.selection_clear(0, END)
        playlist.activate(nextSong)
        playlist.selection_set(nextSong, last=None)
    else:
        playlist.selection_clear(0, END)
        playlist.activate(0)
        playlist.selection_set(0, last=None)

    # play the next song
    playableNextSong = f'song-folder/{playlist.get(ACTIVE)}.mp3'
    pygame.mixer.music.load(playableNextSong)
    pygame.mixer.music.play(loops=0)


    setSongDisplay(playlist.get(ACTIVE))
    global isPaused
    isPaused = False

    
def playLast():
    """
    Function to play the previous track.
    """
    # select next song in song box, select last in list if at start
    if(playlist.get(ACTIVE) != playlist.get(0)):
        lastSong = playlist.curselection()
        lastSong = lastSong[0]-1
        playlist.selection_clear(0, END)
        playlist.activate(lastSong)
        playlist.selection_set(lastSong, last=None)
    else:
        playlist.selection_clear(0, END)
        playlist.activate(END)
        playlist.selection_set(END, last=None)
        
    # play the previous song
    playableNextSong = f'song-folder/{playlist.get(ACTIVE)}.mp3'
    pygame.mixer.music.load(playableNextSong)
    pygame.mixer.music.play(loops=0)
    
    setSongDisplay(playlist.get(ACTIVE))
    global isPaused
    isPaused = False
    
    
def setSongDisplay(currentSong):
    """
    Function to set album art, song title, and album name displays.
    """
    global root
    global albumArtwork
    global songTitle
    global artistName
    
    # Load audio file and default artwork
    thisSong = f'song-folder/{currentSong}.mp3'
    audiofile = eyed3.load(thisSong)
    albumName = audiofile.tag.album
    albumPhoto = Image.open(f'song-folder/art-default')
    returnedImage = ImageTk.PhotoImage(albumPhoto)

    # find the corresponding album art file found earlier
    song_list = os.listdir("song-folder/")
    for file in song_list:
        if (file.endswith(f'{albumName}.jpg')):
            albumPhoto = Image.open(f'song-folder/{file}')
            returnedImage = ImageTk.PhotoImage(albumPhoto)
    
    # set and display artwork, artist, and title
    currentSongTitle = str(audiofile.tag.title)
    currentArtistName = str(audiofile.tag.artist)
    albumArtwork['image'] = returnedImage
    songTitle['text'] = currentSongTitle
    artistName['text'] = currentArtistName
    root.mainloop()


def main():
    """
    The main function for the MP3 Python Music Player.
    """
    getAlbumArt()
    build_program()

if __name__ == "__main__":
    main()